In [1]:
##Libraries
#Dataframe & Arrays
import pandas as pd
import numpy as np

df = pd.read_csv('../data/cleaned_df2.csv')

In [10]:
df = df.sort_values(by=['StockCode','UnitPrice'], ascending=False)

In [12]:
df_pricing_file = df.groupby(['StockCode','UnitPrice']).agg({'Rev': ['sum','count'],'Quantity':['sum']}).reset_index()
df_pricing_file = df_pricing_file.sort_values(by=['StockCode','UnitPrice'], ascending=False)

StockCode UnitPrice    Rev       Quantity
                            sum count      sum
7287    90214Z      0.29   3.48     1       12
7286    90214Y      1.25   1.25     1        1
7285    90214Y      0.29  17.40     2       60
7284    90214W      0.29   3.48     1       12
7283    90214V      1.25   2.50     2        2

In [15]:
df_pricing_file.columns = ['StockCode','UnitPrice','Rev','Count','Qty']
df_pricing_file.head()

,StockCode,UnitPrice,Rev,Count,Qty
0,10002,0.85,699.55,49,823
1,10080,0.39,112.71,20,289
2,10080,0.85,1.70,1,2
3,10120,0.21,40.53,30,193
4,10123C,0.65,3.25,3,5


In [20]:
df_awp = df_pricing_file.groupby(['StockCode']).agg({'Rev': ['sum'],'Qty':['sum']})
df_awp.columns = ['Rev','Qty']

In [21]:
df_awp.head()

,Rev,Qty
StockCode,,
10002,699.55,823
10080,114.41,291
10120,40.53,193
10123C,3.25,5
10124A,6.72,16


In [22]:
def AWP(row):
    return (row['Rev'] / row['Qty'])

In [23]:
df_awp['AWP'] = (df_awp['Rev'] / df_awp['Qty'])
df_awp.head()

,Rev,Qty,AWP
StockCode,,,
10002,699.55,823,0.850000
10080,114.41,291,0.393162
10120,40.53,193,0.210000
10123C,3.25,5,0.650000
10124A,6.72,16,0.420000


In [47]:
df_awp.reset_index(inplace=True)
df_awp.columns  = ['StockCode','Rev','Qty','AWP']
df_awp.head()

,StockCode,Rev,Qty,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10123C,3.25,5,0.650000
4,10124A,6.72,16,0.420000


In [ ]:
## trying to identify mode of Qty purchased by StockCode

In [52]:
df_qty_mode = df.groupby(['StockCode'])['Quantity'].mean().astype(int).to_frame()
df_qty_mode.reset_index(inplace=True)
df_qty_mode.columns  = ['StockCode','Quantity']

In [53]:
df_qty_mode.head()

,StockCode,Quantity
0,10002,16
1,10080,13
2,10120,6
3,10123C,1
4,10124A,3


In [57]:
def qtymodelookup(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding mode of quantity sold for that item
    
    The purpose of this funtion is to vlookup value of item from a reference dataframe (df_qty_mode)
    """
    try:
        return int(df_qty_mode.loc[df_qty_mode['StockCode'] == row]["Quantity"])
    except:
        #print(f"Failed on row {row}")
        pass

In [58]:
df_awp['ModeQty'] = df_awp['StockCode'].apply(qtymodelookup)

In [59]:
df_awp.head()

,StockCode,Rev,Qty,AWP,ModeQty
0,10002,699.55,823,0.850000,16
1,10080,114.41,291,0.393162,13
2,10120,40.53,193,0.210000,6
3,10123C,3.25,5,0.650000,1
4,10124A,6.72,16,0.420000,3


In [60]:
df_awp.to_csv('../data/df_awp2.csv', index=True)